In [1]:
import os
os.chdir("../../crystal/")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.io as pio

from tqdm import tqdm
from math import lcm
from typing import List, Dict
from ripser import Rips
from utils import wasserstein_distance_matrix, plot_distance_matrix

from Symmetry import Symmetry
from UnitCell import UnitCell
from RandomCrystal import RandomCrystal
from Fractional import FractionalCoordinate, FractionalCoordinateList
from Positional import PositionalCoordinate, PositionalCoordinateList

from sklearn.manifold import TSNE, Isomap, MDS, SpectralEmbedding
from sklearn.multiclass import OneVsOneClassifier
from sklearn.svm import SVC

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

plt.rcParams['text.usetex'] = False

In [2]:
df = pd.read_csv("output/space_groups.csv", sep=" ")
df

,Space Group,Crystal System,Asymmetric Unit,Symmetries,Group Order,Unit Cell
0,2,Triclinic,30,"x,y,z;-x,-y,-z",2,"13.027,11.1822,9.2189,92.35,107.3,104.5"
1,9,Monoclinic,71,"x,y,z;x,-y,z+1/2;x+1/2,y+1/2,z;x+1/2,-y+1/2,z+1/2",4,"20.408,13.304,20.166,90.0,102.33,90.0"
2,10,Monoclinic,50,"x,y,z;-x,y,-z;-x,-y,-z;x,-y,z",4,"12.7411,12.6989,20.9991,90.0,96.29,90.0"
3,11,Monoclinic,25,"x,y,z;-x,y+1/2,-z;-x,-y,-z;x,-y+1/2,z",4,"11.454,21.695,7.227,90.0,93.15,90.0"
4,12,Monoclinic,24,"x,y,z;-x,y,-z;-x,-y,-z;x,-y,z;x+1/2,y+1/2,z;-x...",8,"22.684,13.373,12.553,90.0,69.48,90.0"
...,...,...,...,...,...,...
68,223,Cubic,7,"x,y,z;-x,-y,z;x,-y,-z;-x,y,-z;z,x,y;y,z,x;-z,-...",48,"13.705,13.705,13.705,90.0,90.0,90.0"
69,225,Cubic,4,"x,y,z;-x,-y,z;x,-y,-z;-x,y,-z;z,x,y;y,z,x;-z,-...",192,"13.624,13.624,13.624,90.0,90.0,90.0"
70,227,Cubic,5,"x,y,z;-x+1/4,-y+1/4,z;x,-y+1/4,-z+1/4;-x+1/4,y...",192,"24.345,24.345,24.345,90.0,90.0,90.0"
71,229,Cubic,5,"x,y,z;-x,-y,z;x,-y,-z;-x,y,-z;z,x,y;y,z,x;-z,-...",96,"18.578,18.578,18.578,90.0,90.0,90.0"


# Monoclinic vs Orthorhombic

In [3]:
crystal_system_1: str = 'Monoclinic'
crystal_system_2: str = 'Orthorhombic'

space_groups_1: np.ndarray = df.loc[df['Crystal System'] == crystal_system_1]['Space Group'].unique()
space_groups_2: np.ndarray = df.loc[df['Crystal System'] == crystal_system_2]['Space Group'].unique()

random_space_group_index: np.ndarray = np.random.randint(low = 0, high = len(space_groups_1), size = 1, dtype = int)[0]
space_group_1: int = space_groups_1[random_space_group_index]

random_space_group_index: np.ndarray = np.random.randint(low = 0, high = len(space_groups_2), size = 1, dtype = int)[0]
space_group_2: int = space_groups_2[random_space_group_index]

row_1 = df.loc[df['Space Group'] == space_group_1]
row_2 = df.loc[df['Space Group'] == space_group_2]

group_order_1: int = row_1['Group Order'].values[0]
group_order_2: int = row_2['Group Order'].values[0]

least_common_multiple: int = lcm(group_order_1, group_order_2)
print(f'Least common multiple: {least_common_multiple}')

crystal_dict = {}
n = 48

Least common multiple: 16


In [4]:
symmetries: list = row_1['Symmetries'].values[0].split(sep=";")
symmetries: List[Symmetry] = [Symmetry(sym) for sym in symmetries]

unit_cell: List[str] = row_1['Unit Cell'].values[0].split(sep=",")
unit_cell: UnitCell = UnitCell(*[float(x) for x in unit_cell])
normalised_unit_cell: UnitCell = unit_cell.normalise()

normalising_constant: float = unit_cell.normalising_constant
k = least_common_multiple // group_order_1
if least_common_multiple < n:
    k *= (n // least_common_multiple)

rips = Rips(maxdim=2, verbose=False)

for _ in tqdm(range(50)):
    random_crystal: RandomCrystal = RandomCrystal(symmetries, k)
    positional_coordinates: PositionalCoordinateList = random_crystal.fractional_coordinates.orthogonalise(unit_cell)
    normalised_positional_coordinates: PositionalCoordinateList = positional_coordinates.normalise(normalising_constant)

    distance_matrix: np.ndarray = normalised_positional_coordinates.calculate_distance_matrix(normalised_unit_cell, boundary_conditions=True)
    persistence = rips.fit_transform(X = distance_matrix, distance_matrix=True)

    for dim, intervals in enumerate(persistence):
        persistence[dim] = np.array(list(filter(lambda i: i[1] < float('inf'), intervals)))

    crystal_dict[f'{space_group_1}_{_+1}'] = {
        'system': crystal_system_1,
        'persistence': persistence
    }

100%|██████████| 50/50 [00:00<00:00, 72.39it/s]


In [5]:
symmetries: list = row_2['Symmetries'].values[0].split(sep=";")
symmetries: List[Symmetry] = [Symmetry(sym) for sym in symmetries]

unit_cell: List[str] = row_2['Unit Cell'].values[0].split(sep=",")
unit_cell: UnitCell = UnitCell(*[float(x) for x in unit_cell])
normalised_unit_cell: UnitCell = unit_cell.normalise()

normalising_constant: float = unit_cell.normalising_constant
k = least_common_multiple // group_order_2
if least_common_multiple < n:
    k *= (n // least_common_multiple)

rips = Rips(maxdim=2, verbose=False)

for _ in tqdm(range(50)):
    random_crystal: RandomCrystal = RandomCrystal(symmetries, k)
    positional_coordinates: PositionalCoordinateList = random_crystal.fractional_coordinates.orthogonalise(unit_cell)
    normalised_positional_coordinates: PositionalCoordinateList = positional_coordinates.normalise(normalising_constant)

    distance_matrix: np.ndarray = normalised_positional_coordinates.calculate_distance_matrix(normalised_unit_cell, boundary_conditions=True)
    persistence = rips.fit_transform(X = distance_matrix, distance_matrix=True)

    for dim, intervals in enumerate(persistence):
        persistence[dim] = np.array(list(filter(lambda i: i[1] < float('inf'), intervals)))

    crystal_dict[f'{space_group_2}_{_+1}'] = {
        'system': crystal_system_2,
        'persistence': persistence
    }

100%|██████████| 50/50 [00:00<00:00, 60.01it/s]


In [6]:
distance_matrix_0 = wasserstein_distance_matrix(crystal_dict, 0)
distance_matrix_1 = wasserstein_distance_matrix(crystal_dict, 1)
distance_matrix_2 = wasserstein_distance_matrix(crystal_dict, 2)

distance_matrix: np.ndarray = np.maximum.reduce([distance_matrix_0, distance_matrix_1, distance_matrix_2])

100%|██████████| 100/100 [00:01<00:00, 86.91it/s]


In [7]:
system = [v['system'] for v in crystal_dict.values()]
crystals = list(crystal_dict.keys())

n = 5
mds = MDS(n_components=n, dissimilarity='precomputed', metric=True)
embedding = mds.fit_transform(distance_matrix)

embedding_df = pd.DataFrame(embedding, columns=[str(_+1) for _ in range(n)])
embedding_df['Crystal System'] = system
embedding_df['Name'] = crystals

X = embedding
y = embedding_df['Crystal System'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

embedding_df.head()

,1,2,3,4,5,Crystal System,Name
0,-0.775253,0.491980,-0.229569,0.015623,-0.520967,Monoclinic,12_1
1,-0.728834,0.156534,-0.092243,0.672801,0.249527,Monoclinic,12_2
2,-0.151225,-0.135885,-0.021377,0.530866,-0.220262,Monoclinic,12_3
3,0.699772,0.518443,0.301742,-0.128093,-0.296676,Monoclinic,12_4
4,0.476755,0.457183,-0.174947,0.812912,-0.663901,Monoclinic,12_5


In [8]:
clf = OneVsOneClassifier(SVC())
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

class_report = classification_report(y_test, y_pred, target_names = embedding_df['Crystal System'].unique())
print('Classification Report:')
print(class_report)

Accuracy: 0.90
Classification Report:
              precision    recall  f1-score   support

  Monoclinic       1.00      0.82      0.90        17
Orthorhombic       0.81      1.00      0.90        13

    accuracy                           0.90        30
   macro avg       0.91      0.91      0.90        30
weighted avg       0.92      0.90      0.90        30



# Monoclinic vs Tetragonal

In [9]:
crystal_system_1: str = 'Monoclinic'
crystal_system_2: str = 'Tetragonal'

space_groups_1: np.ndarray = df.loc[df['Crystal System'] == crystal_system_1]['Space Group'].unique()
space_groups_2: np.ndarray = df.loc[df['Crystal System'] == crystal_system_2]['Space Group'].unique()

random_space_group_index: np.ndarray = np.random.randint(low = 0, high = len(space_groups_1), size = 1, dtype = int)[0]
space_group_1: int = space_groups_1[random_space_group_index]

random_space_group_index: np.ndarray = np.random.randint(low = 0, high = len(space_groups_2), size = 1, dtype = int)[0]
space_group_2: int = space_groups_2[random_space_group_index]

row_1 = df.loc[df['Space Group'] == space_group_1]
row_2 = df.loc[df['Space Group'] == space_group_2]

group_order_1: int = row_1['Group Order'].values[0]
group_order_2: int = row_2['Group Order'].values[0]

least_common_multiple: int = lcm(group_order_1, group_order_2)
print(f'Least common multiple: {least_common_multiple}')

crystal_dict = {}
n = 48

Least common multiple: 16


In [10]:
symmetries: list = row_1['Symmetries'].values[0].split(sep=";")
symmetries: List[Symmetry] = [Symmetry(sym) for sym in symmetries]

unit_cell: List[str] = row_1['Unit Cell'].values[0].split(sep=",")
unit_cell: UnitCell = UnitCell(*[float(x) for x in unit_cell])
normalised_unit_cell: UnitCell = unit_cell.normalise()

normalising_constant: float = unit_cell.normalising_constant
k = least_common_multiple // group_order_1
if least_common_multiple < n:
    k *= (n // least_common_multiple)

rips = Rips(maxdim=2, verbose=False)

for _ in tqdm(range(50)):
    random_crystal: RandomCrystal = RandomCrystal(symmetries, k)
    positional_coordinates: PositionalCoordinateList = random_crystal.fractional_coordinates.orthogonalise(unit_cell)
    normalised_positional_coordinates: PositionalCoordinateList = positional_coordinates.normalise(normalising_constant)

    distance_matrix: np.ndarray = normalised_positional_coordinates.calculate_distance_matrix(normalised_unit_cell, boundary_conditions=True)
    persistence = rips.fit_transform(X = distance_matrix, distance_matrix=True)

    for dim, intervals in enumerate(persistence):
        persistence[dim] = np.array(list(filter(lambda i: i[1] < float('inf'), intervals)))

    crystal_dict[f'{space_group_1}_{_+1}'] = {
        'system': crystal_system_1,
        'persistence': persistence
    }

100%|██████████| 50/50 [00:00<00:00, 77.50it/s]


In [11]:
symmetries: list = row_2['Symmetries'].values[0].split(sep=";")
symmetries: List[Symmetry] = [Symmetry(sym) for sym in symmetries]

unit_cell: List[str] = row_2['Unit Cell'].values[0].split(sep=",")
unit_cell: UnitCell = UnitCell(*[float(x) for x in unit_cell])
normalised_unit_cell: UnitCell = unit_cell.normalise()

normalising_constant: float = unit_cell.normalising_constant
k = least_common_multiple // group_order_2
if least_common_multiple < n:
    k *= (n // least_common_multiple)

rips = Rips(maxdim=2, verbose=False)

for _ in tqdm(range(50)):
    random_crystal: RandomCrystal = RandomCrystal(symmetries, k)
    positional_coordinates: PositionalCoordinateList = random_crystal.fractional_coordinates.orthogonalise(unit_cell)
    normalised_positional_coordinates: PositionalCoordinateList = positional_coordinates.normalise(normalising_constant)

    distance_matrix: np.ndarray = normalised_positional_coordinates.calculate_distance_matrix(normalised_unit_cell, boundary_conditions=True)
    persistence = rips.fit_transform(X = distance_matrix, distance_matrix=True)

    for dim, intervals in enumerate(persistence):
        persistence[dim] = np.array(list(filter(lambda i: i[1] < float('inf'), intervals)))

    crystal_dict[f'{space_group_2}_{_+1}'] = {
        'system': crystal_system_2,
        'persistence': persistence
    }

100%|██████████| 50/50 [00:01<00:00, 47.58it/s]


In [12]:
distance_matrix_0 = wasserstein_distance_matrix(crystal_dict, 0)
distance_matrix_1 = wasserstein_distance_matrix(crystal_dict, 1)
distance_matrix_2 = wasserstein_distance_matrix(crystal_dict, 2)

distance_matrix: np.ndarray = np.maximum.reduce([distance_matrix_0, distance_matrix_1, distance_matrix_2])

100%|██████████| 100/100 [00:01<00:00, 76.41it/s]


In [13]:
system = [v['system'] for v in crystal_dict.values()]
crystals = list(crystal_dict.keys())

n = 5
mds = MDS(n_components=n, dissimilarity='precomputed', metric=True)
embedding = mds.fit_transform(distance_matrix)

embedding_df = pd.DataFrame(embedding, columns=[str(_+1) for _ in range(n)])
embedding_df['Crystal System'] = system
embedding_df['Name'] = crystals

X = embedding
y = embedding_df['Crystal System'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

embedding_df.head()

,1,2,3,4,5,Crystal System,Name
0,0.230137,-0.199955,0.376172,-0.752661,0.090166,Monoclinic,9_1
1,-0.025234,-0.047683,0.481706,-0.277336,0.295642,Monoclinic,9_2
2,-0.524304,-0.288643,0.208370,0.154317,-0.263510,Monoclinic,9_3
3,-0.506586,0.008863,0.321086,-0.815695,-0.410417,Monoclinic,9_4
4,-0.082540,0.107985,0.023237,-0.444339,0.597437,Monoclinic,9_5


In [14]:
clf = OneVsOneClassifier(SVC())
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

class_report = classification_report(y_test, y_pred, target_names = embedding_df['Crystal System'].unique())
print('Classification Report:')
print(class_report)

Accuracy: 0.93
Classification Report:
              precision    recall  f1-score   support

  Monoclinic       1.00      0.88      0.94        17
  Tetragonal       0.87      1.00      0.93        13

    accuracy                           0.93        30
   macro avg       0.93      0.94      0.93        30
weighted avg       0.94      0.93      0.93        30



# Monoclinic vs Trigonal

In [21]:
crystal_system_1: str = 'Monoclinic'
crystal_system_2: str = 'Trigonal'

space_groups_1: np.ndarray = df.loc[df['Crystal System'] == crystal_system_1]['Space Group'].unique()
space_groups_2: np.ndarray = df.loc[df['Crystal System'] == crystal_system_2]['Space Group'].unique()

random_space_group_index: np.ndarray = np.random.randint(low = 0, high = len(space_groups_1), size = 1, dtype = int)[0]
space_group_1: int = space_groups_1[random_space_group_index]

random_space_group_index: np.ndarray = np.random.randint(low = 0, high = len(space_groups_2), size = 1, dtype = int)[0]
space_group_2: int = space_groups_2[random_space_group_index]

row_1 = df.loc[df['Space Group'] == space_group_1]
row_2 = df.loc[df['Space Group'] == space_group_2]

group_order_1: int = row_1['Group Order'].values[0]
group_order_2: int = row_2['Group Order'].values[0]

least_common_multiple: int = lcm(group_order_1, group_order_2)
print(f'Least common multiple: {least_common_multiple}')

crystal_dict = {}
n = 48

Least common multiple: 72


In [23]:
symmetries: list = row_1['Symmetries'].values[0].split(sep=";")
symmetries: List[Symmetry] = [Symmetry(sym) for sym in symmetries]

unit_cell: List[str] = row_1['Unit Cell'].values[0].split(sep=",")
unit_cell: UnitCell = UnitCell(*[float(x) for x in unit_cell])
normalised_unit_cell: UnitCell = unit_cell.normalise()

normalising_constant: float = unit_cell.normalising_constant
k = least_common_multiple // group_order_1
if least_common_multiple < n:
    k *= (n // least_common_multiple)

rips = Rips(maxdim=2, verbose=False)

for _ in tqdm(range(50)):
    random_crystal: RandomCrystal = RandomCrystal(symmetries, k)
    positional_coordinates: PositionalCoordinateList = random_crystal.fractional_coordinates.orthogonalise(unit_cell)
    normalised_positional_coordinates: PositionalCoordinateList = positional_coordinates.normalise(normalising_constant)

    distance_matrix: np.ndarray = normalised_positional_coordinates.calculate_distance_matrix(normalised_unit_cell, boundary_conditions=True)
    persistence = rips.fit_transform(X = distance_matrix, distance_matrix=True)

    for dim, intervals in enumerate(persistence):
        persistence[dim] = np.array(list(filter(lambda i: i[1] < float('inf'), intervals)))

    crystal_dict[f'{space_group_1}_{_+1}'] = {
        'system': crystal_system_1,
        'persistence': persistence
    }

  0%|          | 0/50 [00:00<?, ?it/s]

100%|██████████| 50/50 [00:01<00:00, 25.11it/s]


In [24]:
symmetries: list = row_2['Symmetries'].values[0].split(sep=";")
symmetries: List[Symmetry] = [Symmetry(sym) for sym in symmetries]

unit_cell: List[str] = row_2['Unit Cell'].values[0].split(sep=",")
unit_cell: UnitCell = UnitCell(*[float(x) for x in unit_cell])
normalised_unit_cell: UnitCell = unit_cell.normalise()

normalising_constant: float = unit_cell.normalising_constant
k = least_common_multiple // group_order_2
if least_common_multiple < n:
    k *= (n // least_common_multiple)

rips = Rips(maxdim=2, verbose=False)

for _ in tqdm(range(50)):
    random_crystal: RandomCrystal = RandomCrystal(symmetries, k)
    positional_coordinates: PositionalCoordinateList = random_crystal.fractional_coordinates.orthogonalise(unit_cell)
    normalised_positional_coordinates: PositionalCoordinateList = positional_coordinates.normalise(normalising_constant)

    distance_matrix: np.ndarray = normalised_positional_coordinates.calculate_distance_matrix(normalised_unit_cell, boundary_conditions=True)
    persistence = rips.fit_transform(X = distance_matrix, distance_matrix=True)

    for dim, intervals in enumerate(persistence):
        persistence[dim] = np.array(list(filter(lambda i: i[1] < float('inf'), intervals)))

    crystal_dict[f'{space_group_2}_{_+1}'] = {
        'system': crystal_system_2,
        'persistence': persistence
    }

100%|██████████| 50/50 [00:02<00:00, 19.08it/s]


In [25]:
distance_matrix_0 = wasserstein_distance_matrix(crystal_dict, 0)
distance_matrix_1 = wasserstein_distance_matrix(crystal_dict, 1)
distance_matrix_2 = wasserstein_distance_matrix(crystal_dict, 2)

distance_matrix: np.ndarray = np.maximum.reduce([distance_matrix_0, distance_matrix_1, distance_matrix_2])

100%|██████████| 100/100 [00:01<00:00, 69.26it/s]


In [26]:
system = [v['system'] for v in crystal_dict.values()]
crystals = list(crystal_dict.keys())

n = 5
mds = MDS(n_components=n, dissimilarity='precomputed', metric=True)
embedding = mds.fit_transform(distance_matrix)

embedding_df = pd.DataFrame(embedding, columns=[str(_+1) for _ in range(n)])
embedding_df['Crystal System'] = system
embedding_df['Name'] = crystals

X = embedding
y = embedding_df['Crystal System'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

embedding_df.head()

,1,2,3,4,5,Crystal System,Name
0,-0.512560,0.614708,-1.241839,0.266248,-0.421273,Monoclinic,15_1
1,-0.657537,0.369298,-0.959566,-0.153510,0.403191,Monoclinic,15_2
2,-0.691478,-0.082155,-0.089305,0.361073,0.465768,Monoclinic,15_3
3,-0.115820,-0.014040,1.044064,0.785292,-0.322464,Monoclinic,15_4
4,-0.114091,0.808100,-0.867878,0.685158,0.396137,Monoclinic,15_5


In [27]:
clf = OneVsOneClassifier(SVC())
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

class_report = classification_report(y_test, y_pred, target_names = embedding_df['Crystal System'].unique())
print('Classification Report:')
print(class_report)

Accuracy: 0.97
Classification Report:
              precision    recall  f1-score   support

  Monoclinic       0.94      1.00      0.97        17
    Trigonal       1.00      0.92      0.96        13

    accuracy                           0.97        30
   macro avg       0.97      0.96      0.97        30
weighted avg       0.97      0.97      0.97        30



# Monoclinic vs Hexagonal

In [35]:
crystal_system_1: str = 'Monoclinic'
crystal_system_2: str = 'Hexagonal'

space_groups_1: np.ndarray = df.loc[df['Crystal System'] == crystal_system_1]['Space Group'].unique()
space_groups_2: np.ndarray = df.loc[df['Crystal System'] == crystal_system_2]['Space Group'].unique()

random_space_group_index: np.ndarray = np.random.randint(low = 0, high = len(space_groups_1), size = 1, dtype = int)[0]
space_group_1: int = space_groups_1[random_space_group_index]

random_space_group_index: np.ndarray = np.random.randint(low = 0, high = len(space_groups_2), size = 1, dtype = int)[0]
space_group_2: int = space_groups_2[random_space_group_index]

row_1 = df.loc[df['Space Group'] == space_group_1]
row_2 = df.loc[df['Space Group'] == space_group_2]

group_order_1: int = row_1['Group Order'].values[0]
group_order_2: int = row_2['Group Order'].values[0]

least_common_multiple: int = lcm(group_order_1, group_order_2)
print(f'Least common multiple: {least_common_multiple}')

crystal_dict = {}
n = 48

Least common multiple: 24


In [36]:
symmetries: list = row_1['Symmetries'].values[0].split(sep=";")
symmetries: List[Symmetry] = [Symmetry(sym) for sym in symmetries]

unit_cell: List[str] = row_1['Unit Cell'].values[0].split(sep=",")
unit_cell: UnitCell = UnitCell(*[float(x) for x in unit_cell])
normalised_unit_cell: UnitCell = unit_cell.normalise()

normalising_constant: float = unit_cell.normalising_constant
k = least_common_multiple // group_order_1
if least_common_multiple < n:
    k *= (n // least_common_multiple)

rips = Rips(maxdim=2, verbose=False)

for _ in tqdm(range(50)):
    random_crystal: RandomCrystal = RandomCrystal(symmetries, k)
    positional_coordinates: PositionalCoordinateList = random_crystal.fractional_coordinates.orthogonalise(unit_cell)
    normalised_positional_coordinates: PositionalCoordinateList = positional_coordinates.normalise(normalising_constant)

    distance_matrix: np.ndarray = normalised_positional_coordinates.calculate_distance_matrix(normalised_unit_cell, boundary_conditions=True)
    persistence = rips.fit_transform(X = distance_matrix, distance_matrix=True)

    for dim, intervals in enumerate(persistence):
        persistence[dim] = np.array(list(filter(lambda i: i[1] < float('inf'), intervals)))

    crystal_dict[f'{space_group_1}_{_+1}'] = {
        'system': crystal_system_1,
        'persistence': persistence
    }

100%|██████████| 50/50 [00:00<00:00, 61.16it/s]


In [37]:
symmetries: list = row_2['Symmetries'].values[0].split(sep=";")
symmetries: List[Symmetry] = [Symmetry(sym) for sym in symmetries]

unit_cell: List[str] = row_2['Unit Cell'].values[0].split(sep=",")
unit_cell: UnitCell = UnitCell(*[float(x) for x in unit_cell])
normalised_unit_cell: UnitCell = unit_cell.normalise()

normalising_constant: float = unit_cell.normalising_constant
k = least_common_multiple // group_order_2
if least_common_multiple < n:
    k *= (n // least_common_multiple)

rips = Rips(maxdim=2, verbose=False)

for _ in tqdm(range(50)):
    random_crystal: RandomCrystal = RandomCrystal(symmetries, k)
    positional_coordinates: PositionalCoordinateList = random_crystal.fractional_coordinates.orthogonalise(unit_cell)
    normalised_positional_coordinates: PositionalCoordinateList = positional_coordinates.normalise(normalising_constant)

    distance_matrix: np.ndarray = normalised_positional_coordinates.calculate_distance_matrix(normalised_unit_cell, boundary_conditions=True)
    persistence = rips.fit_transform(X = distance_matrix, distance_matrix=True)

    for dim, intervals in enumerate(persistence):
        persistence[dim] = np.array(list(filter(lambda i: i[1] < float('inf'), intervals)))

    crystal_dict[f'{space_group_2}_{_+1}'] = {
        'system': crystal_system_2,
        'persistence': persistence
    }

100%|██████████| 50/50 [00:01<00:00, 47.48it/s]


In [38]:
distance_matrix_0 = wasserstein_distance_matrix(crystal_dict, 0)
distance_matrix_1 = wasserstein_distance_matrix(crystal_dict, 1)
distance_matrix_2 = wasserstein_distance_matrix(crystal_dict, 2)

distance_matrix: np.ndarray = np.maximum.reduce([distance_matrix_0, distance_matrix_1, distance_matrix_2])

100%|██████████| 100/100 [00:01<00:00, 87.87it/s]


In [39]:
system = [v['system'] for v in crystal_dict.values()]
crystals = list(crystal_dict.keys())

n = 5
mds = MDS(n_components=n, dissimilarity='precomputed', metric=True)
embedding = mds.fit_transform(distance_matrix)

embedding_df = pd.DataFrame(embedding, columns=[str(_+1) for _ in range(n)])
embedding_df['Crystal System'] = system
embedding_df['Name'] = crystals

X = embedding
y = embedding_df['Crystal System'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

embedding_df.head()

,1,2,3,4,5,Crystal System,Name
0,0.450592,0.167632,0.206459,0.705532,0.626213,Monoclinic,9_1
1,0.006509,0.072198,-0.141758,0.878599,-0.342972,Monoclinic,9_2
2,0.772370,-0.086290,0.943204,0.480953,0.427038,Monoclinic,9_3
3,-0.246274,0.267198,-0.090661,0.332684,-0.292561,Monoclinic,9_4
4,-0.335904,-0.038503,0.180145,0.856510,0.196812,Monoclinic,9_5


In [40]:
clf = OneVsOneClassifier(SVC())
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

class_report = classification_report(y_test, y_pred, target_names = embedding_df['Crystal System'].unique())
print('Classification Report:')
print(class_report)

Accuracy: 0.97
Classification Report:
              precision    recall  f1-score   support

  Monoclinic       0.93      1.00      0.96        13
   Hexagonal       1.00      0.94      0.97        17

    accuracy                           0.97        30
   macro avg       0.96      0.97      0.97        30
weighted avg       0.97      0.97      0.97        30



# Monoclinic vs Cubic

In [41]:
crystal_system_1: str = 'Monoclinic'
crystal_system_2: str = 'Cubic'

space_groups_1: np.ndarray = df.loc[df['Crystal System'] == crystal_system_1]['Space Group'].unique()
space_groups_2: np.ndarray = df.loc[df['Crystal System'] == crystal_system_2]['Space Group'].unique()

random_space_group_index: np.ndarray = np.random.randint(low = 0, high = len(space_groups_1), size = 1, dtype = int)[0]
space_group_1: int = space_groups_1[random_space_group_index]

random_space_group_index: np.ndarray = np.random.randint(low = 0, high = len(space_groups_2), size = 1, dtype = int)[0]
space_group_2: int = space_groups_2[random_space_group_index]

row_1 = df.loc[df['Space Group'] == space_group_1]
row_2 = df.loc[df['Space Group'] == space_group_2]

group_order_1: int = row_1['Group Order'].values[0]
group_order_2: int = row_2['Group Order'].values[0]

least_common_multiple: int = lcm(group_order_1, group_order_2)
print(f'Least common multiple: {least_common_multiple}')

crystal_dict = {}
n = 48

Least common multiple: 96


In [42]:
symmetries: list = row_1['Symmetries'].values[0].split(sep=";")
symmetries: List[Symmetry] = [Symmetry(sym) for sym in symmetries]

unit_cell: List[str] = row_1['Unit Cell'].values[0].split(sep=",")
unit_cell: UnitCell = UnitCell(*[float(x) for x in unit_cell])
normalised_unit_cell: UnitCell = unit_cell.normalise()

normalising_constant: float = unit_cell.normalising_constant
k = least_common_multiple // group_order_1
if least_common_multiple < n:
    k *= (n // least_common_multiple)

rips = Rips(maxdim=2, verbose=False)

for _ in tqdm(range(50)):
    random_crystal: RandomCrystal = RandomCrystal(symmetries, k)
    positional_coordinates: PositionalCoordinateList = random_crystal.fractional_coordinates.orthogonalise(unit_cell)
    normalised_positional_coordinates: PositionalCoordinateList = positional_coordinates.normalise(normalising_constant)

    distance_matrix: np.ndarray = normalised_positional_coordinates.calculate_distance_matrix(normalised_unit_cell, boundary_conditions=True)
    persistence = rips.fit_transform(X = distance_matrix, distance_matrix=True)

    for dim, intervals in enumerate(persistence):
        persistence[dim] = np.array(list(filter(lambda i: i[1] < float('inf'), intervals)))

    crystal_dict[f'{space_group_1}_{_+1}'] = {
        'system': crystal_system_1,
        'persistence': persistence
    }

100%|██████████| 50/50 [00:04<00:00, 11.56it/s]


In [43]:
symmetries: list = row_2['Symmetries'].values[0].split(sep=";")
symmetries: List[Symmetry] = [Symmetry(sym) for sym in symmetries]

unit_cell: List[str] = row_2['Unit Cell'].values[0].split(sep=",")
unit_cell: UnitCell = UnitCell(*[float(x) for x in unit_cell])
normalised_unit_cell: UnitCell = unit_cell.normalise()

normalising_constant: float = unit_cell.normalising_constant
k = least_common_multiple // group_order_2
if least_common_multiple < n:
    k *= (n // least_common_multiple)

rips = Rips(maxdim=2, verbose=False)

for _ in tqdm(range(50)):
    random_crystal: RandomCrystal = RandomCrystal(symmetries, k)
    positional_coordinates: PositionalCoordinateList = random_crystal.fractional_coordinates.orthogonalise(unit_cell)
    normalised_positional_coordinates: PositionalCoordinateList = positional_coordinates.normalise(normalising_constant)

    distance_matrix: np.ndarray = normalised_positional_coordinates.calculate_distance_matrix(normalised_unit_cell, boundary_conditions=True)
    persistence = rips.fit_transform(X = distance_matrix, distance_matrix=True)

    for dim, intervals in enumerate(persistence):
        persistence[dim] = np.array(list(filter(lambda i: i[1] < float('inf'), intervals)))

    crystal_dict[f'{space_group_2}_{_+1}'] = {
        'system': crystal_system_2,
        'persistence': persistence
    }

100%|██████████| 50/50 [00:09<00:00,  5.24it/s]


In [44]:
distance_matrix_0 = wasserstein_distance_matrix(crystal_dict, 0)
distance_matrix_1 = wasserstein_distance_matrix(crystal_dict, 1)
distance_matrix_2 = wasserstein_distance_matrix(crystal_dict, 2)

distance_matrix: np.ndarray = np.maximum.reduce([distance_matrix_0, distance_matrix_1, distance_matrix_2])

100%|██████████| 100/100 [00:03<00:00, 26.69it/s]


In [45]:
system = [v['system'] for v in crystal_dict.values()]
crystals = list(crystal_dict.keys())

n = 5
mds = MDS(n_components=n, dissimilarity='precomputed', metric=True)
embedding = mds.fit_transform(distance_matrix)

embedding_df = pd.DataFrame(embedding, columns=[str(_+1) for _ in range(n)])
embedding_df['Crystal System'] = system
embedding_df['Name'] = crystals

X = embedding
y = embedding_df['Crystal System'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

embedding_df.head()

,1,2,3,4,5,Crystal System,Name
0,-0.039644,0.161578,-0.432109,-0.817307,0.834220,Monoclinic,9_1
1,0.591387,-0.167386,-0.267927,-0.599779,0.840153,Monoclinic,9_2
2,0.011751,-0.375347,0.326015,-0.126027,0.539519,Monoclinic,9_3
3,0.018180,-0.427665,-0.442267,-0.866687,0.182212,Monoclinic,9_4
4,0.550965,-0.393688,-1.284181,-1.031077,0.562738,Monoclinic,9_5


In [46]:
clf = OneVsOneClassifier(SVC())
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

class_report = classification_report(y_test, y_pred, target_names = embedding_df['Crystal System'].unique())
print('Classification Report:')
print(class_report)

Accuracy: 1.00
Classification Report:
              precision    recall  f1-score   support

  Monoclinic       1.00      1.00      1.00        13
       Cubic       1.00      1.00      1.00        17

    accuracy                           1.00        30
   macro avg       1.00      1.00      1.00        30
weighted avg       1.00      1.00      1.00        30

